**Task 09: Data linking**

In [16]:
!pip install rdflib
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4/"

In [17]:
from rdflib import Graph, Namespace, Literal, URIRef
g1 = Graph()
g2 = Graph()
g3 = Graph()
g1.parse(github_storage+"resources/data03.rdf", format="xml")
g2.parse(github_storage+"resources/data04.rdf", format="xml")

<Graph identifier=Ncf06929bd4c74267947ac6a7e5b519c2 (<class 'rdflib.graph.Graph'>)>

Search for individuals in the two graphs and link them using the OWL: sameAs property, insert these matches in g3. We consider that two individuals are the same if they have the same nickname and family name. Keep in mind that the URIs do not have to be the same for the same individual in the two graphs.

1. All individuals in the two graphs

1.1. Individuals in g1

In [18]:
ns1 = Namespace("http://data.three.org#")
from rdflib.plugins.sparql import prepareQuery

q11 = prepareQuery('''
  SELECT 
    ?s
  WHERE { 
    ?s a/rdfs:subClassOf* ns:Person. 
  }
  ''',
  initNs = { "ns": ns1}
)

for r in g1.query(q11):
  print(r)

(rdflib.term.URIRef('http://data.three.org#SaraJones'),)
(rdflib.term.URIRef('http://data.three.org#JohnDoe'),)
(rdflib.term.URIRef('http://data.three.org#JohnSmith'),)
(rdflib.term.URIRef('http://data.three.org#HarryPotter'),)


1.2. Individuals in g2

In [19]:
ns2 = Namespace("http://data.four.org#")

q12 = prepareQuery('''
  SELECT 
    ?s
  WHERE { 
    ?s a/rdfs:subClassOf* ns:Person. 
  }
  ''',
  initNs = { "ns": ns2}
)

for r in g2.query(q12):
  print(r)

(rdflib.term.URIRef('http://data.four.org#0004'),)
(rdflib.term.URIRef('http://data.four.org#0005'),)
(rdflib.term.URIRef('http://data.four.org#0001'),)
(rdflib.term.URIRef('http://data.four.org#0003'),)
(rdflib.term.URIRef('http://data.four.org#0002'),)


1.3. To have more clarity in the individuals of g2:

In [20]:
vcard = Namespace("http://www.w3.org/2001/vcard-rdf/3.0#")

q13 = prepareQuery('''
  SELECT 
    ?s ?name
  WHERE { 
    ?s a/rdfs:subClassOf* ns:Person. 
    ?s vcard:FN ?name
  }
  ''',
  initNs = { "ns": ns2, "vcard": vcard}
)

for r in g2.query(q13):
  print(r.name)

Tim Berners Lee
John Doe
Sara Jones
Harry Potter
John Smith


2. Link them using the OWL: sameAs property, insert these matches in g3 (two individuals are the same if they have the same nickname and family name)

2.1. Nickname (vcard:Given) and Family (vcard:Family) from g1

In [21]:
from rdflib.namespace import RDF, RDFS
q21 = prepareQuery('''
  SELECT 
    ?s ?family ?nickname
  WHERE { 
    ?s a/rdfs:subClassOf* ns:Person. 
    ?s vcard:Given ?nickname.
    ?s vcard:Family ?family
  }
  ''',
  initNs = { "ns": ns1, "vcard": vcard}
)

for r in g1.query(q21):
  print(r)

(rdflib.term.URIRef('http://data.three.org#SaraJones'), rdflib.term.Literal('Jones', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')), rdflib.term.Literal('Sara', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://data.three.org#JohnDoe'), rdflib.term.Literal('Doe', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')), rdflib.term.Literal('John', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://data.three.org#JohnSmith'), rdflib.term.Literal('Smith', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')), rdflib.term.Literal('John', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))


TAKE INTO ACCOUNT: the individual called "Harry Potter" has no "Family" property, so it would not be matched in g3

2.2. Nickname(vcard:Given) and Family(vcard:Family) from g2

In [22]:
q22 = prepareQuery('''
  SELECT 
    ?s ?family ?nickname
  WHERE { 
    ?s a/rdfs:subClassOf* ns:Person. 
    ?s vcard:Given ?nickname.
    ?s vcard:Family ?family
  }
  ''',
  initNs = { "ns": ns2, "vcard": vcard}
)

for r in g2.query(q22):
  print(r)

(rdflib.term.URIRef('http://data.four.org#0004'), rdflib.term.Literal('Berners Lee', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')), rdflib.term.Literal('Tim', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://data.four.org#0005'), rdflib.term.Literal('Doe', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')), rdflib.term.Literal('John', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://data.four.org#0001'), rdflib.term.Literal('Jones', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')), rdflib.term.Literal('Sara', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://data.four.org#0003'), rdflib.term.Literal('Potter', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')), rdflib.term.Literal('Harry', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLS

2.3. Match the results in g3

In [23]:
from rdflib.namespace import OWL
for r1 in g1.query(q21):
  for r2 in g2.query(q22):
    if r1.family == r2.family and r1.nickname == r2.nickname:
      g3.add((r1.s, OWL.sameAs, r2.s))
      print((r1.s, OWL.sameAs, r2.s))

(rdflib.term.URIRef('http://data.three.org#SaraJones'), rdflib.term.URIRef('http://www.w3.org/2002/07/owl#sameAs'), rdflib.term.URIRef('http://data.four.org#0001'))
(rdflib.term.URIRef('http://data.three.org#JohnDoe'), rdflib.term.URIRef('http://www.w3.org/2002/07/owl#sameAs'), rdflib.term.URIRef('http://data.four.org#0005'))


TAKE INTO ACCOUNT: John Smith has different nicknames, so it is not matched